In [1]:
pip install tensorflow

  Using cached wheel-0.44.0-py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/390.3 MB 8.4 MB/s eta 0:00:47
   ---------------------------------------- 3.1/390.3 MB 8.8 MB/s eta 0:00:45
    --------------------------------------- 5.5/390.3 MB 9.6 MB/s eta 0:00:41
    --------------------------------------- 6.8/390.3 MB 8.9 MB/s eta 0:00:43
    --------------------------------------- 7.3/390.3 MB 8.0 MB/s eta 0:00:49
    --------------------------------------- 8.1/390.3 MB 6.9 MB/s eta 0:00:56
    --------------------------------------- 8.4/390.3 MB 6.3 MB/s eta 0:01:01
    --------------------------------------- 8.9/390.3 MB 5.5 MB/s eta 0:01:10
    --------------------------------------- 9.2/390.3 MB 5.0 MB/s eta 0:01:17
    --------------------------------------- 9.2/390.3 MB 5.0 MB/s eta 0:01:17
    --------------------------------------- 9.4/390.3 MB 4.5 MB/s eta 0:01:25
    ----

In [3]:
import pandas as pd

In [2]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

In [8]:
df = pd.read_csv('data/ratings.csv')
df.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [5]:
from sklearn.model_selection import train_test_split

In [7]:
traindf, testdf = train_test_split(df, test_size = 0.2, random_state = 42)
traindf.shape

(785404, 3)

In [17]:
n_books = df["book_id"].nunique()
n_books

10000

In [18]:
n_users = df["user_id"].nunique()
n_users

53424

В первую очередь нам необходимо создать эмбеддинги для книг и пользователей. Создаём эмбеддинги для книг:

In [19]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

Сначала мы задаём размерность входного слоя. После этого определяем размер эмбеддинга — в данном случае снижаем размерность до 5. Далее мы разворачиваем результат в массив с одним измерением с помощью слоя Flatten().

Делаем то же самое для пользователей:

In [20]:
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

Теперь, когда мы создали представления как для книг, так и для пользователей, нам необходимо соединить их:

In [21]:
conc = Concatenate()([book_vec, user_vec])

Далее начинаем «собирать» нашу нейронную сеть из слоёв. Dense обозначает полносвязный слой. Также мы обозначаем для него количество нейронов и данные, которые идут на вход.

In [22]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

Собираем модель — передаём входные данные для книг и пользователей, а также архитектуру нейронной сети:

In [23]:
model2 = Model([user_input, book_input], out)

Также нам необходимо задать алгоритм оптимизации и метрику, которую мы будем оптимизировать. В данном случае будем использовать метод adam и хорошо известную вам среднеквадратичную ошибку. Adam - вариция градиентного спуска

In [24]:
model2.compile(optimizer = 'adam',loss =  'mean_squared_error')

В параметр эпох передаём значение 5: у нас будет реализовано пять эпох — пять обучений нейронной сети. На каждой из эпох обновляются веса для минимизации ошибки.

In [26]:
history = model2.fit([traindf.user_id, traindf.book_id], traindf.rating, epochs=5, verbose=1)

Epoch 1/5


c:\Users\Настенька\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['User-Input', 'Book-Input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


24544/24544 ━━━━━━━━━━━━━━━━━━━━ 70s 3ms/step - loss: 1.0074
Epoch 2/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 68s 3ms/step - loss: 0.6830
Epoch 3/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 65s 3ms/step - loss: 0.6570
Epoch 4/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 65s 3ms/step - loss: 0.6297
Epoch 5/5
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 66s 3ms/step - loss: 0.6101


Теперь можно оценить качество:

In [28]:
model2.evaluate([testdf.user_id, testdf.book_id], testdf.rating)

  40/6136 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - loss: 0.6569     

c:\Users\Настенька\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['User-Input', 'Book-Input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


6136/6136 ━━━━━━━━━━━━━━━━━━━━ 4s 711us/step - loss: 0.7070


0.7070751190185547

Обычно для улучшения качества модели каким-то образом модифицируют нейронную сеть: дополняют её, увеличивают время обучения. Добавим ещё один полносвязный слой с восемью нейронами после полносвязного слоя с 32 нейронами. Обучим нейронную сеть, реализовав десять эпох:

In [29]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([traindf.user_id, traindf.book_id], traindf.rating, epochs=10, verbose=1)
model2.evaluate([testdf.user_id, testdf.book_id], testdf.rating)

Epoch 1/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 74s 3ms/step - loss: 0.7603
Epoch 2/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 71s 3ms/step - loss: 0.5699
Epoch 3/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 69s 3ms/step - loss: 0.5444
Epoch 4/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 64s 3ms/step - loss: 0.5242
Epoch 5/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 70s 3ms/step - loss: 0.5113
Epoch 6/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 80s 3ms/step - loss: 0.4964
Epoch 7/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 73s 3ms/step - loss: 0.4879
Epoch 8/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 73s 3ms/step - loss: 0.4806
Epoch 9/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 74s 3ms/step - loss: 0.4718
Epoch 10/10
24544/24544 ━━━━━━━━━━━━━━━━━━━━ 74s 3ms/step - loss: 0.4662
6136/6136 ━━━━━━━━━━━━━━━━━━━━ 4s 679us/step - loss: 0.7720


0.7722353339195251